In [1]:
import agentpy as ap
import matplotlib.pyplot as plt
import seaborn as sns
import IPython
import random

In [105]:
from math import ceil
from subprocess import call

def chooseMove():
     return random.randrange(-1, 2, 1)

class IntersectionModel(ap.Model):

    def setup(self):

        spawn_point_posible = [(ceil(self.p.sizeM/2),0),(0,ceil(self.p.sizeN/2))]
        spawn_point = [random.choice(spawn_point_posible) for i in range(self.p['num_agentes'])]

        # Agentes
        self.coches = self.agents = ap.AgentList(self, self.p['num_agentes'])

        celdas_totales_H = self.p.sizeN * 3 
        celdas_totales_V = self.p.sizeM * 3
        self.calleH = ap.AgentList(self,celdas_totales_H)
        self.calleV = ap.AgentList(self,celdas_totales_V)
    
        calle_vertical1 = [(n, int(self.p.sizeN/2) - 1) for n in range(0, self.p.sizeM, 1)]
        calle_vertical2 = [(n, int(self.p.sizeN/2)) for n in range(0, self.p.sizeM, 1)]
        calle_vertical3 = [(n, int(self.p.sizeN/2) + 1) for n in range(0, self.p.sizeM, 1)]

        calle_horizontal1 = [(int(self.p.sizeM/2) - 1, n) for n in range(0, self.p.sizeN, 1)]
        calle_horizontal2 = [(int(self.p.sizeM/2), n) for n in range(0, self.p.sizeN, 1)]
        calle_horizontal3 = [(int(self.p.sizeM/2) + 1, n) for n in range(0, self.p.sizeN, 1)]

        calleHorizontal = calle_horizontal1 + calle_horizontal2 + calle_horizontal3
        calleVertical = calle_vertical1 + calle_vertical2 + calle_vertical3

        # Grid Piso
        self.floor = ap.Grid(self, (self.p.sizeM,self.p.sizeN), track_empty=True, torus=True)
        self.floor.add_agents(self.coches,spawn_point,random=False, empty=True)
        self.floor.add_agents(self.calleH,calleHorizontal,random=False,empty=True)
        self.floor.add_agents(self.calleV,calleVertical,random=False,empty=True)
        
        # Initiate a dynamic variable for all trees
        # Condition 0: Carro, 1: CalleHorizontal, 2: CalleVertical
        self.coches.condition = 0
        self.calleH.condition = 1
        self.calleV.condition = 2
                              
    def step(self):

        for carro in self.coches:
            for neighbor in self.floor.neighbors(carro):
                if neighbor.condition == 1 and neighbor.condition == 2:
                    self.floor.move_by(carro,[0,1])
                    break
                elif neighbor.condition == 1 and neighbor.condition != 2:
                    self.floor.move_by(carro,[0,1])
                    break
                elif neighbor.condition == 2 and neighbor.condition != 1:
                    self.floor.move_by(carro,[1,0])
                    break
            else:
                self.floor.move_by(carro,[0,0])


In [106]:
parameters = {
    'num_agentes': 50,
    'sizeM': 40, #Ancho
    'sizeN': 50, #Largo
    'steps': 100,
}

In [107]:
# Create single-run animation with custom colors

def animation_plot(model, ax):
    attr_grid = model.floor.attr_grid('condition')
    color_dict = {0:'#FF0000', 1:'#808080', 2:'#808080', None:'#00A300'}
    #0:'#744700', 1:'#3d9de6', 2:'#bcbcbc', None:'#ffffff'
    ap.gridplot(attr_grid, ax=ax, color_dict=color_dict, convert=True)
    ax.set_title(f"Simulation limpieza de celdas\n"
                 f"Time-step: {model.t}")

fig, ax = plt.subplots()
model = IntersectionModel(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml(fps=15))